In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
sys.path.insert(0, '/cndd/fangming/CEMBA/scripts/ClusterCrossValidation')

from __init__ import *
from __init__jupyterlab import *
import seaborn as sns
set_matplotlib_formats('retina')
import importlib
import snmcseq_utils
importlib.reload(snmcseq_utils)
from matplotlib import collections as mc


import pickle
import xarray
import fbpca
from sklearn.model_selection import KFold
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import r2_score
import itertools
from scipy import sparse
from scipy.stats import spearmanr
from scipy import optimize
from scipy.optimize import root_scalar
from scipy import stats
from statsmodels.stats.multitest import multipletests

import CEMBA_run_tsne
import CEMBA_clst_utils
import cluster_cv_utils
importlib.reload(cluster_cv_utils)
from cluster_cv_utils import *

import gseapy

In [2]:
f = '/cndd2/fangming/references/gencode.vM16.annotation.gtf.gz'
df = pd.read_csv(f, sep='\t', skiprows=5, header=None)
print(df.shape)
df.head()

(1771112, 9)


0        1           2        3        4  5  6  7  \
0  chr1   HAVANA        gene  3073253  3074322  .  +  .   
1  chr1   HAVANA  transcript  3073253  3074322  .  +  .   
2  chr1   HAVANA        exon  3073253  3074322  .  +  .   
3  chr1  ENSEMBL        gene  3102016  3102125  .  +  .   
4  chr1  ENSEMBL  transcript  3102016  3102125  .  +  .   

                                                   8  
0  gene_id "ENSMUSG00000102693.1"; gene_type "TEC...  
1  gene_id "ENSMUSG00000102693.1"; transcript_id ...  
2  gene_id "ENSMUSG00000102693.1"; transcript_id ...  
3  gene_id "ENSMUSG00000064842.1"; gene_type "snR...  
4  gene_id "ENSMUSG00000064842.1"; transcript_id ...

In [3]:
df[2].unique()

array(['gene', 'transcript', 'exon', 'CDS', 'start_codon', 'stop_codon',
       'UTR', 'Selenocysteine'], dtype=object)

In [4]:
df = df[df[2]=='transcript']
print(df.shape)
df.head()

(133849, 9)


0        1           2        3        4  5  6  7  \
1   chr1   HAVANA  transcript  3073253  3074322  .  +  .   
4   chr1  ENSEMBL  transcript  3102016  3102125  .  +  .   
7   chr1   HAVANA  transcript  3205901  3216344  .  -  .   
10  chr1   HAVANA  transcript  3206523  3215632  .  -  .   
13  chr1   HAVANA  transcript  3214482  3671498  .  -  .   

                                                    8  
1   gene_id "ENSMUSG00000102693.1"; transcript_id ...  
4   gene_id "ENSMUSG00000064842.1"; transcript_id ...  
7   gene_id "ENSMUSG00000051951.5"; transcript_id ...  
10  gene_id "ENSMUSG00000051951.5"; transcript_id ...  
13  gene_id "ENSMUSG00000051951.5"; transcript_id ...

In [6]:
chromosomes = np.hstack([['chr{}'.format(i+1) for i in range(19)], ['chrX']])
print(chromosomes)

df = df[df[0].isin(chromosomes)]

['chr1' 'chr2' 'chr3' 'chr4' 'chr5' 'chr6' 'chr7' 'chr8' 'chr9' 'chr10'
 'chr11' 'chr12' 'chr13' 'chr14' 'chr15' 'chr16' 'chr17' 'chr18' 'chr19'
 'chrX']


In [7]:
df.iloc[0, 8]

'gene_id "ENSMUSG00000102693.1"; transcript_id "ENSMUST00000193812.1"; gene_type "TEC"; gene_name "4933401J01Rik"; transcript_type "TEC"; transcript_name "4933401J01Rik-201"; level 2; transcript_support_level "NA"; tag "basic"; havana_gene "OTTMUSG00000049935.1"; havana_transcript "OTTMUST00000127109.1";'

In [8]:
df['gene_id'] = df[8].apply(lambda x: x.split(';')[0].split('"')[1])
df['transcript_id'] = df[8].apply(lambda x: x.split(';')[1].split('"')[1])
df['gene_name'] = df[8].apply(lambda x: x.split(';')[3].split('"')[1])
df['transcript_name'] = df[8].apply(lambda x: x.split(';')[5].split('"')[1])

In [9]:
df.head()

0        1           2        3        4  5  6  7  \
1   chr1   HAVANA  transcript  3073253  3074322  .  +  .   
4   chr1  ENSEMBL  transcript  3102016  3102125  .  +  .   
7   chr1   HAVANA  transcript  3205901  3216344  .  -  .   
10  chr1   HAVANA  transcript  3206523  3215632  .  -  .   
13  chr1   HAVANA  transcript  3214482  3671498  .  -  .   

                                                    8               gene_id  \
1   gene_id "ENSMUSG00000102693.1"; transcript_id ...  ENSMUSG00000102693.1   
4   gene_id "ENSMUSG00000064842.1"; transcript_id ...  ENSMUSG00000064842.1   
7   gene_id "ENSMUSG00000051951.5"; transcript_id ...  ENSMUSG00000051951.5   
10  gene_id "ENSMUSG00000051951.5"; transcript_id ...  ENSMUSG00000051951.5   
13  gene_id "ENSMUSG00000051951.5"; transcript_id ...  ENSMUSG00000051951.5   

           transcript_id      gene_name    transcript_name  
1   ENSMUST00000193812.1  4933401J01Rik  4933401J01Rik-201  
4   ENSMUST00000082908.1        Gm26206        Gm26206-201  
7   ENSMUST00000162897.1           Xkr4           Xkr4-203  
10  ENSMUST00000159265.1           Xkr4           Xkr4-202  
13  ENSMUST00000070533.4           Xkr4           Xkr4-201

In [10]:
df_out = df[[0, 3, 4, 6, 'transcript_id', 'transcript_name', 'gene_id', 'gene_name']]
print(df_out.shape)
df_out.head()

(131568, 8)


0        3        4  6         transcript_id    transcript_name  \
1   chr1  3073253  3074322  +  ENSMUST00000193812.1  4933401J01Rik-201   
4   chr1  3102016  3102125  +  ENSMUST00000082908.1        Gm26206-201   
7   chr1  3205901  3216344  -  ENSMUST00000162897.1           Xkr4-203   
10  chr1  3206523  3215632  -  ENSMUST00000159265.1           Xkr4-202   
13  chr1  3214482  3671498  -  ENSMUST00000070533.4           Xkr4-201   

                 gene_id      gene_name  
1   ENSMUSG00000102693.1  4933401J01Rik  
4   ENSMUSG00000064842.1        Gm26206  
7   ENSMUSG00000051951.5           Xkr4  
10  ENSMUSG00000051951.5           Xkr4  
13  ENSMUSG00000051951.5           Xkr4

In [11]:
f = '/cndd2/fangming/references/gencode_vM16_transcript_annotation.bed'
df_out.to_csv(f, sep='\t', header=False, index=False)